In [3]:
import pandas as pd
import numpy as np
import scipy
import math
import scipy.stats
from sklearn.metrics import mean_squared_error
import textract
import os
import pickle
from pathlib import Path
import numpy as np
import random
import pandas as pd
from itertools import combinations
root = Path(".")

In [4]:
my_path = root / "Pickled_files" / "item_similarity_matrix"
dbfile = open(my_path, 'rb')     
item_sim = pickle.load(dbfile)
dbfile.close()

In [5]:
rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')
rating_data.columns = ['UserID', 'ItemID', 'Rating', 'Timestamp']
rating_data.drop(columns=['Timestamp'], axis=1, inplace=True)

C:\Users\ashwi\AppData\Local\Temp\ipykernel_2352\1128856203.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')


In [6]:
rating_data

,UserID,ItemID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [7]:
item_cnt = max(rating_data['ItemID']) + 1
users_cnt = max(rating_data['UserID']) + 1
item_cnt, users_cnt

(3953, 6041)

In [8]:
rating_data.shape

(1000209, 3)

In [9]:
rating_train = rating_data.sample(frac=0.8, random_state=200)
rating_test = rating_data.drop(rating_train.index)

In [10]:
matrix = np.zeros(shape=(users_cnt, item_cnt))
matrix.shape

(6041, 3953)

In [11]:
for row in rating_train.itertuples():
    matrix[row.UserID][row.ItemID] = row.Rating
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.]])

In [12]:
item_sim.shape

(3953, 3953)

In [13]:
def get_sim_items(itemID) : 
    sim_userID = list()
    for item in range(1 , item_sim.shape[0]) : 
        if item == itemID : 
            continue
        sim_userID.append((item_sim[itemID][item] , item))
    sim_userID.sort(key=lambda y: -y[0])
    return sim_userID

In [14]:
sim_top_k = dict()
for i in range(1 , item_cnt) : 
    sim_top_k[i] = get_sim_items(i)

In [15]:
def predict(user_id , item_id , K) : 
    sim_items = sim_top_k[item_id]
    pred_rating , sum_sim = 0 , 0
    taken = 0
    for (item_s , itemID) in sim_items :
        if item_s < 0 or math.isnan(item_s): 
            continue
        if taken == K :
            break
        if matrix[user_id][itemID] > 0 : 
            taken += 1
            sum_sim += item_s   
            pred_rating += item_s * matrix[user_id][itemID]
    if sum_sim == 0 : 
        return sum_sim
    a = pred_rating / sum_sim
    return pred_rating / sum_sim

In [16]:
for user in range(1 , users_cnt) : 
    print(user)
    for item in range(1 , item_cnt) : 
        if matrix[user][item] == 0 :
            matrix[user][item] = predict(user , item , 5)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [17]:
for i in matrix : 
    print(i)

[0. 0. 0. ... 0. 0. 0.]
[0.         4.8205123  4.39766519 ... 4.70378116 4.73979053 4.68459535]
[0.         4.13735754 3.         ... 3.27905294 3.47972922 3.34977903]
[0.         3.81384229 3.7409472  ... 3.93275481 3.89914709 3.92793396]
[0.         4.01537258 4.28256935 ... 4.19726942 4.16339528 4.20759036]
[0.         3.60089381 3.11921158 ... 2.95641225 3.30876684 3.05308604]
[0.         4.         3.81119937 ... 4.1613071  4.1090607  4.16822161]
[0.         4.26966974 4.36507717 ... 4.50052365 4.47726373 4.47124149]
[0.         3.36602092 3.35379018 ... 3.82622983 3.72950916 3.87612811]
[0.         4.26521143 3.23529928 ... 3.92437522 3.98193424 3.9086028 ]
[0.         4.62925113 4.44726006 ... 4.21867808 4.3050116  4.31033925]
[0.         3.15727865 2.74771192 ... 3.2356831  3.15307027 3.22437641]
[0.         3.53110547 3.61933891 ... 3.80908952 3.74351596 3.81376912]
[0.         3.26404185 3.         ... 2.78126867 2.80251679 2.80278888]
[0.         2.70109753 2.54757178 ... 3.

In [22]:
def rmse_matrix(mat, test_data):
    y_actual = list(test_data.Rating)
    y_pred = list()
    for id , row in test_data.iterrows() : 
        uid , itid = row['UserID'] , row['ItemID']
        y_pred.append(mat[uid][itid])
    return mean_squared_error(y_actual , y_pred)

In [23]:
print(rmse_matrix(matrix , rating_test))

1.1558117182207637
